# Görselden Mermaid Diyagramı

In [ ]:
import io
import json
import ipywidgets as widgets
from IPython.display import display, Markdown
import pytesseract
from PIL import Image
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
def translate_to_turkish(text: str) -> str:
    if not text.strip():
        return ""
    messages = [
        SystemMessage(content="Sen metinleri Türkçeye çeviren bir çeviri asistanısın."),
        HumanMessage(content=text),
    ]
    return llm.invoke(messages).content.strip()

In [ ]:
def diagram_from_text(text: str) -> dict:
    prompt = (
        "Aşağıdaki metne göre diyagramın türünü belirle, "
        "kısa bir Türkçe açıklama yaz ve buna uygun Mermaid kodu üret. "
        "Sonucu JSON olarak {type, description, mermaid} formatında ver.\n"
        f"Metin: {text}"
    )
    messages = [HumanMessage(content=prompt)]
    try:
        content = llm.invoke(messages).content.strip()
        return json.loads(content)
    except Exception:
        return {"type": "", "description": "", "mermaid": ""}

In [ ]:
uploader = widgets.FileUpload(accept='image/*', multiple=False)
display(uploader)

In [ ]:
def handle_upload(change):
    if uploader.value:
        item = list(uploader.value.values())[0]
        image = Image.open(io.BytesIO(item['content']))
        display(image)
        extracted = pytesseract.image_to_string(image)
        turkish = translate_to_turkish(extracted)
        result = diagram_from_text(turkish)
        display(Markdown(f"### Diyagram Türü\n{result.get('type', '')}"))
        display(Markdown(f"### Açıklama\n{result.get('description', '')}"))
        code = result.get('mermaid', '')
        display(Markdown("### Mermaid Kod"))
        display(Markdown(f"```mermaid\n{code}\n```"))
uploader.observe(handle_upload, names='value')